In [472]:
import requests
import json
import numpy as np
from functools import reduce
from tqdm import *

In [462]:
def get_result(url):
    header = { 'X-API-KEY':"gXhJxaWsNLUTfL9UeGwak4hf1F8fcunI7AR7HfCV"}
    r = requests.get(url = url, headers = header)
    try:
        data = r.json()['results']
    except Exception:
        print("Error getting data")
        return None
    return data

def generate_urls_recent_votes(number):
    urls = []
    offset = 0
    for i in range(0, number):
        url = "https://api.propublica.org/congress/v1/house/votes/recent.json?offset={}".format(i * 20)
        urls.append(url)
    return np.array(urls)

In [463]:
# GET BILL URLS
urls = generate_urls_recent_votes(500) # 10000 BILLS
urls[:-10]

array(['https://api.propublica.org/congress/v1/house/votes/recent.json?offset=0',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=20',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=40',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=60',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=80',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=100',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=120',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=140',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=160',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=180',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=200',
       'https://api.propublica.org/congress/v1/house/votes/recent.json?offset=220'

In [464]:
recent_votes_results = [get_result(url) for url in urls]
removed_nones_recent_votes_results = [x for x in recent_votes_results if x is not None]
recent_votes_results_flat = [vote for votes in removed_nones_recent_votes_results for vote in votes['votes']]

In [476]:
def get_bill(vote):
    bill_uri = vote['bill']['api_uri']
    if bill_uri is None:
        return None
    bill = get_result(bill_uri)[0]
    if bill is not None:
        return bill

In [477]:
# PROCESS VOTES

def process_vote(vote):    
    bill = get_bill(vote)
    if bill is None:
        return None, None
    
    bill_slug = bill['bill_slug']
    bill_summary = bill['summary']
    bill_date = vote['date']
    bill_type = bill['bill_type']
    congress = bill['congress']
    bill_uri = bill['bill_uri']
    subjects = bill['primary_subject']
    
    cols = ['bill_slug', 'summary', 'date', 'bill_type', 'congress', 'bill_uri', 'subjects']
    values = [[bill_slug, bill_summary,bill_date,bill_type, congress, bill_uri, subjects]]
    bill_row_df = pd.DataFrame(values, columns=cols)
    
    actual_vote = get_result(vote['vote_uri'])
    positions = actual_vote['votes']['vote']['positions']
    
    vote_df = pd.DataFrame(positions)
    vote_df = vote_df.rename(columns={'vote_position': bill_slug}).drop('dw_nominate', 1)
    return vote_df, bill_row_df

In [478]:
# HACK TO CREATE DUMMY TABLE - SKETCH
vote_df, bill_row = process_vote(recent_votes_results_flat[0])
vote_df.drop(vote_df.columns[len(vote_df.columns)-1], axis=1, inplace=True)
vote_df 

,district,member_id,name,party,state
0,5,A000374,Ralph Abraham,R,LA
1,12,A000370,Alma Adams,D,NC
2,4,A000055,Robert B. Aderholt,R,AL
3,31,A000371,Pete Aguilar,D,CA
4,12,A000372,Rick Allen,R,GA
5,3,A000367,Justin Amash,R,MI
6,2,A000369,Mark Amodei,R,NV
7,19,A000375,Jodey Arrington,R,TX
8,36,B001291,Brian Babin,R,TX
9,2,B001298,Don Bacon,R,NE


In [480]:
votes_df = vote_df
bills_df = pd.DataFrame()
count = 0
for vote in tqdm(recent_votes_results_flat):
    try:
        vote_df, bill_row = process_vote(vote)
        votes_df = pd.merge(votes_df, vote_df, on=['member_id', 'name', 'district', 'state', 'party'], how='outer')
        bills_df = bills_df.append(bill_row)
    except:
        continue


100%|██████████| 9600/9600 [2:01:18<00:00,  1.40it/s]  


In [469]:
votes_df

,district,member_id,name,party,state
0,5,A000374,Ralph Abraham,R,LA
1,12,A000370,Alma Adams,D,NC
2,4,A000055,Robert B. Aderholt,R,AL
3,31,A000371,Pete Aguilar,D,CA
4,12,A000372,Rick Allen,R,GA
5,3,A000367,Justin Amash,R,MI
6,2,A000369,Mark Amodei,R,NV
7,19,A000375,Jodey Arrington,R,TX
8,36,B001291,Brian Babin,R,TX
9,2,B001298,Don Bacon,R,NE


In [471]:
votes_df.to_csv('./voting_data_2000.csv')
bills_df.to_csv('./bill_data_2000.csv')